# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [17]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests


# Import API key
from api_keys import geoapify_key

In [18]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,lere,10.3870,8.5726,22.75,24,23,3.86,NG,1706647970
1,1,iskateley,67.6803,53.1512,-6.16,93,76,4.59,RU,1706647970
2,2,thompson,55.7435,-97.8558,4.09,75,75,3.60,CA,1706647970
3,3,verkhniye tatyshly,56.2912,55.8575,-4.28,97,100,2.65,RU,1706647970
4,4,tura,25.5198,90.2201,13.04,50,0,1.65,IN,1706647970


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [37]:
%%capture --no-display

# Configure the map plot

humidity = city_data_df["Humidity"]

# Display the map
# YOUR CODE HERE
map_plot = city_data_df.hvplot.points(
"Lng",
"Lat",
geo = True,
tiles = "OSM",
frame_width = 600,
frame_height = 550,
size = "Humidity",
color = "City",
alpha= 0.75)

map_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [50]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df.dropna()
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21)]
city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] == 0)]
city_data_df

# Drop any rows with null values


# Display sample data


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
116,116,port elizabeth,-33.9180,25.5701,22.40,84,0,1.54,ZA,1706647985
153,153,cidreira,-30.1811,-50.2056,24.38,69,0,5.94,BR,1706647991
163,163,timbuktu,16.7735,-3.0074,22.06,13,0,5.34,ML,1706647993
167,167,guerrero negro,27.9769,-114.0611,22.88,36,0,4.47,MX,1706647994
195,195,nouadhibou,20.9310,-17.0347,23.99,33,0,4.63,MR,1706647999
225,225,nova sintra,14.8667,-24.7167,21.16,66,0,5.38,CV,1706648004
253,253,lompoc,34.6391,-120.4579,23.72,51,0,0.89,US,1706647831
260,260,geraldton,-28.7667,114.6000,21.23,78,0,4.63,AU,1706648008
280,280,wickenburg,33.9686,-112.7296,23.99,24,0,2.30,US,1706648011
305,305,laguna,38.4210,-121.4238,23.08,68,0,4.12,US,1706648015


### Step 3: Create a new DataFrame called `hotel_df`.

In [42]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = pd.DataFrame(city_data_df, columns=["City", "Country", "Lat", "Lng", "Humidity"])

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df["Hotel Name"] = ""

# Display sample data
# YOUR CODE HERE
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,lere,NG,10.3870,8.5726,24,
1,iskateley,RU,67.6803,53.1512,93,
2,thompson,CA,55.7435,-97.8558,75,
3,verkhniye tatyshly,RU,56.2912,55.8575,97,
4,tura,IN,25.5198,90.2201,50,
...,...,...,...,...,...,...
590,batang,ID,-6.4846,110.7083,87,
591,broken hill,AU,-31.9500,141.4333,48,
592,zajecar,RS,43.9036,22.2641,74,
593,tevriz,RU,57.5090,72.4042,97,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [45]:
# Set parameters to search for a hotel
radius = 10000
params = {
   
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    # YOUR CODE HERE
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
lere - nearest hotel: No hotel found
iskateley - nearest hotel: Авантаж
thompson - nearest hotel: Thompson Inn
verkhniye tatyshly - nearest hotel: Встреча
tura - nearest hotel: No hotel found
yarmouth - nearest hotel: The Inn At Cape Cod
port-aux-francais - nearest hotel: Keravel
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
waitangi - nearest hotel: Hotel Chathams
kodiak - nearest hotel: Shelikof Lodge
blackmans bay - nearest hotel: Villa Howden
cova figueira - nearest hotel: No hotel found
debre tabor - nearest hotel: Kise alemayehu
tolanaro - nearest hotel: Hôtel Mahavokey
kapuskasing - nearest hotel: Travelodge
aberglasslyn - nearest hotel: Maitland Monte Pio
kaliua - nearest hotel: No hotel found
east london - nearest hotel: No hotel found
sapu padidu - nearest hotel: No hotel found
saint-denis - nearest hotel: Tribe
mugia - nearest hotel: Solpor do Coído
bethel - nearest hotel: Hampton Inn Danbury
fort mcmurray - nearest hotel: Nomad Hotel and Suites

qusmuryn - nearest hotel: No hotel found
vila velha - nearest hotel: Hotel Prainha
sisimiut - nearest hotel: Sømandshjemmet
hilo - nearest hotel: Dolphin Bay Hotel
turpan - nearest hotel: Oriental
nouadhibou - nearest hotel: Hotel valencia
susuman - nearest hotel: No hotel found
belogorsk - nearest hotel: Версаль
west island - nearest hotel: Cocos Village Bungalows
port douglas - nearest hotel: Mantra Aqueous Port Douglas
paranhos - nearest hotel: No hotel found
nova vicosa - nearest hotel: Pousada Pontal da Barra
manay - nearest hotel: No hotel found
moussoro - nearest hotel: No hotel found
nguruka - nearest hotel: Galaxy Lodge
fort st. john - nearest hotel: Fort St. John Motor Inn
baki - nearest hotel: Brother Inn Solo Baru
hamilton - nearest hotel: North Vista Manor
benalla - nearest hotel: Hotel Haven
bilopillya - nearest hotel: No hotel found
san patricio - nearest hotel: No hotel found
kazanskoye - nearest hotel: No hotel found
tabou - nearest hotel: hôtel le rochet
aginskoye - n

ust-nera - nearest hotel: гостиница "Солнечная"
parras de la fuente - nearest hotel: Posada Santa isabel
behshahr - nearest hotel: هتل اشرف
hovd - nearest hotel: Khovd hotel
buka - nearest hotel: No hotel found
papao - nearest hotel: Hiti Moana Villa Lodge
taoudenni - nearest hotel: No hotel found
taroa - nearest hotel: No hotel found
new norfolk - nearest hotel: Woodbridge on the Derwent
richards bay - nearest hotel: SUN1 Richards Bay
ar rass - nearest hotel: Saif Boutique Hotel International
port augusta - nearest hotel: Hotel Commonwealth
berdigestyakh - nearest hotel: No hotel found
yung shue wan - nearest hotel: 浪濤軒酒店 Concerto Inn
karratha - nearest hotel: Karratha International Hotel
cartagena - nearest hotel: Hotel Bellamar
crane - nearest hotel: No hotel found
toliara - nearest hotel: Ambary
santa elena de uairen - nearest hotel: Posada Villa Fairmont
tsiombe - nearest hotel: No hotel found
puerto colombia - nearest hotel: Hotel Puerto Colombia
bonthe - nearest hotel: No hotel 

abadan - nearest hotel: خانه معلم
tantou - nearest hotel: 望江楼
hawera - nearest hotel: White Hart Hotel
bayside - nearest hotel: Hotel Arcata
envira - nearest hotel: No hotel found
grenaa - nearest hotel: Kystvejens Hotel og Konferencecenter
boa vista - nearest hotel: Aipana Plaza Hotel
uriangato - nearest hotel: Hotel Excelencia de America
orivesi - nearest hotel: No hotel found
kittila - nearest hotel: Hotelli Kittilä
tanjung pandan - nearest hotel: Billiton Hotel & Club
duluth - nearest hotel: Radisson Hotel Duluth-Harborview
thai binh - nearest hotel: An Thai Hotel
batang - nearest hotel: Palm Beach Hotel
broken hill - nearest hotel: Ibis Styles
zajecar - nearest hotel: Vila Beba
tevriz - nearest hotel: No hotel found
san buenaventura - nearest hotel: Hotel Casa Bonita


,City,Country,Lat,Lng,Humidity,Hotel Name
0,lere,NG,10.3870,8.5726,24,No hotel found
1,iskateley,RU,67.6803,53.1512,93,Авантаж
2,thompson,CA,55.7435,-97.8558,75,Thompson Inn
3,verkhniye tatyshly,RU,56.2912,55.8575,97,Встреча
4,tura,IN,25.5198,90.2201,50,No hotel found
...,...,...,...,...,...,...
590,batang,ID,-6.4846,110.7083,87,Palm Beach Hotel
591,broken hill,AU,-31.9500,141.4333,48,Ibis Styles
592,zajecar,RS,43.9036,22.2641,74,Vila Beba
593,tevriz,RU,57.5090,72.4042,97,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [47]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
new_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo =  True,
    frame_width = 600,
    frame_height = 400,
    size = "Humidity",
    color = "City",
    alpha = .75,
    hover_cols = ["Hotel Name", "Country"]

)
# Display the map
# YOUR CODE HERE
new_map

:Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)